This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'VtKh9VumYPU5jzy6Fr9n'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [41]:
# First, import the relevant modules
import requests
import collections
import json
import math

In [10]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'
single_day_query = '?limit=1&api_key='

r = requests.get(url + single_day_query + API_KEY)
r_json = json.loads(r.text)
print(r_json)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': 1, 'transform': None, 'column_index': None, 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


In [11]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(r_json, indent=3, sort_keys=True))

{
   "dataset": {
      "collapse": null,
      "column_index": null,
      "column_names": [
         "Date",
         "Open",
         "High",
         "Low",
         "Close",
         "Change",
         "Traded Volume",
         "Turnover",
         "Last Price of the Day",
         "Daily Traded Units",
         "Daily Turnover"
      ],
      "data": [
         [
            "2020-12-01",
            112.2,
            112.2,
            111.5,
            112.0,
            null,
            51.0,
            5703.0,
            null,
            null,
            null
         ]
      ],
      "database_code": "FSE",
      "database_id": 6129,
      "dataset_code": "AFX_X",
      "description": "Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704",
      "end_date": "2020-12-01",
      "frequency": "daily",
      "id": 10095370,
      "limit": 1,
      "name": "Carl Zeiss Meditec (AFX_X)",
  

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
#1 Collect Data from 2017
query_2017 = '?start_date=2017-01-01&end_date=2017-12-31&api_key='
r = requests.get(url + query_2017 + API_KEY)

#2 Convert JSON into Python Dictionary
r_json = json.loads(r.text)
print(type(r_json))

<class 'dict'>


In [23]:
#3 Calculate highest and lowest opening prices for 2017

# Loop through daily data to extract the daily open
opens = []
for day in r_json['dataset']['data']:
    if day[1] != None:
        opens.append(day[1])
    
# Extract & Print Highest and Lowest from 2017
highest = max(opens)
lowest = min(opens)

print("AFX_X's HIGHEST opening price in 2017 was: " + str(highest))
print("AFX_X's LOWEST opening price in 2017 was: " + str(lowest))

AFX_X's HIGHEST opening price in 2017 was: 53.11
AFX_X's LOWEST opening price in 2017 was: 34.0


In [31]:
#4 What was the largest change in any one day (based on High and Low price)?
high = 2
low = 3

# Loop through daily data to extract the intra-day delta
intraday_deltas = []
for day in r_json['dataset']['data']:
    if (day[high] != None) & (day[low] != None):
        delta = day[high] - day[low]
        intraday_deltas.append(delta)
        
# Print Answer
print("AFX_X's largest price change in 2017 was: " + str((max(intraday_deltas))))

AFX_X's largest price change in 2017 was: 2.8100000000000023


In [39]:
#5 What was the largest change between any two days (based on Closing Price)?
close = 4

# Loop through daily data to extract the closing prices
closing_prices = []
for day in r_json['dataset']['data']:
    if day[close] != None:
        closing_prices.append(day[close])
        
# Calculate the Absolute Daily Deltas between days
daily_deltas = []
for i in range(len(closing_prices)):
    if i == 0:
        continue
    
    delta = abs(closing_prices[i] - closing_prices[i-1])
    daily_deltas.append(delta)
    
# Print the answer
max_delta = max(daily_deltas)
print("AFX_X's largest two day price change in 2017 was: " + str(max_delta))

AFX_X's largest two day price change in 2017 was: 2.559999999999995


In [63]:
# Helper Functions for 6 & 7

def mean(lst):
    return sum(lst) / len(lst)

def median(lst):
    lst.sort()
    if len(lst) % 2 > 0:
        index = math.floor(len(lst) / 2)
        return lst[index]
    else:
        index_a = math.floor((len(lst) / 2) -1)
        index_b = math.floor(len(lst) / 2)
        return (lst[index_b] - lst[index_a]) / 2

In [64]:
#6 What was the average daily trading volume during this year?
#7 (Optional) What was the median trading volume during this year?
dv = 6

# Loop through daily data to extract the volume
daily_volumes = []
for day in r_json['dataset']['data']:
    if day[dv] != None:
        daily_volumes.append(day[dv])
        
# Calculate the average & median volumes
mean_vol = mean(daily_volumes)
med_vol = median(daily_volumes)

# Print the answers for 6 & 7
print("AFX_X's average daily trading volume in 2017 was: " + str(mean_vol))
print("AFX_X's median daily trading volume in 2017 was: " + str(med_vol))

AFX_X's average daily trading volume in 2017 was: 89124.33725490196
AFX_X's median daily trading volume in 2017 was: 76286.0
